In [1]:
# dbutils.fs.rm("dbfs:///FileStore/tables/stock/stock_file", True)
# dbutils.fs.rm("dbfs:///FileStore/tables/stock_file", True)

Out[6]: True

In [2]:
from pyspark.sql.types import *
import  pyspark.sql.functions as F

conf = {}
conf["eventhubs.connectionString"] = "......................"

read_df = (
  spark
    .readStream
    .format("eventhubs")
    .options(**conf)
    .load()
)

read_schema = StructType([ 
  StructField("PushERPStockRequest", StructType([
    StructField("UpdateID", StringType(), True),
    StructField("UpdateStockItem", ArrayType(StructType([
      StructField("ProductIdentifier", StructType([
          StructField("SKU", StringType(), True),
          StructField("StoreId", StringType(), True),
       ]), True),
      StructField("InventoryStatus", StringType(), True),
      StructField("AvailableQuantity", StringType(), True),
      StructField("AvailableQuantityActual", StringType(), True), 
     ]) , True ) , True),
  ]), True),  
  StructField("EventProcessedUtcTime", StringType(), True),
  StructField("PartitionId", StringType(), True),
  StructField("EventEnqueuedUtcTime", StringType(), True)
])


decoded_df = read_df.select(F.from_json(F.col("body").cast("string"), read_schema).alias("payload"))

decoded_df.writeStream \
  .format("parquet") \
  .option("path", "dbfs:///FileStore/tables/stock_file") \
    .option("checkpointLocation", "dbfs:///FileStore/tables/stock/stock_file") \
  .start()


Out[7]: <pyspark.sql.streaming.StreamingQuery at 0x7f92d8918e80>